In [ ]:
import pandas as pd
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
import datetime
from packages.parquet_cached_df import ParquetCachedDF

In [ ]:
hr_df = pd.read_csv("weight_loss_dfs/Resting Heart Rate (2).csv")
hr_df = hr_df.rename(columns={"Unnamed: 0": "Date", "Resting Heart Rate": "Resting Heart Rate (bpm)"})

df_path = "./weight_loss_dfs/jordan_df.pqt"
df_manager = ParquetCachedDF(file_path=df_path)
curr_df = df_manager.get_df()

In [ ]:
def get_full_date(date):
    dt = datetime.datetime.strptime(date, "%b %d")
    if dt.month > 8:
        year = 2022
    else:
        year = 2022
    dt = datetime.date(year=year, month=dt.month, day=dt.day)
    return dt.strftime("%Y-%m-%d")
get_full_date("Aug 25")

hr_df["Date"] = hr_df["Date"].apply(get_full_date)
hr_df = hr_df.set_index("Date")
hr_df.index.name = None
hr_df

In [ ]:
curr_df

In [ ]:
joined = curr_df.join(hr_df, how="outer", lsuffix=" curr", rsuffix=" hr")
joined

In [ ]:
import numpy as np
#joined = joined[
#    (joined["Resting Heart Rate (bpm) curr"] != joined["Resting Heart Rate (bpm) hr"]) &
#    ~(np.isnan(joined["Resting Heart Rate (bpm) curr"]) & (np.isnan(joined["Resting Heart Rate (bpm) hr"])))
#]
joined[["Resting Heart Rate (bpm) curr", "Resting Heart Rate (bpm) hr"]]

In [ ]:
def get_hr(row):
    ret = np.nan
    if np.isnan(row["Resting Heart Rate (bpm) curr"]) and np.isnan(row["Resting Heart Rate (bpm) hr"]):
        pass
    elif np.isnan(row["Resting Heart Rate (bpm) curr"]):
        ret = row["Resting Heart Rate (bpm) hr"]
    elif np.isnan(row["Resting Heart Rate (bpm) hr"]):
        ret = row["Resting Heart Rate (bpm) curr"]
    elif row["Resting Heart Rate (bpm) curr"] == row["Resting Heart Rate (bpm) hr"]:
        ret = row["Resting Heart Rate (bpm) curr"]
        print('aggreed')
    else:
        ret = row["Resting Heart Rate (bpm) hr"]
        #raise ValueError(str(row))
    return ret
        
joined["Resting Heart Rate (bpm)"] = joined.apply(get_hr, axis=1)

In [ ]:
joined["Resting Heart Rate (bpm)"]

In [ ]:
for col in joined.columns:
    if col.endswith("curr") or col.endswith("hr"):
        print(col)
joined = joined.drop(columns=["Resting Heart Rate (bpm) curr", "Resting Heart Rate (bpm) hr"])

In [ ]:
joined.shape

In [ ]:
curr_df.shape

In [ ]:
#curr_df.loc["2021-08-24", "BMI"] = 1000
curr_df.reset_index().merge(joined.reset_index(), how='right', indicator=True).shape

In [ ]:
joined.shape

In [ ]:
curr_df.shape

In [ ]:
curr_df.sort_index().index

In [ ]:
joined.sort_index().index

In [ ]:
df_manager.set_df(joined)
df_manager.save_df()

In [ ]:
joined[["Resting Heart Rate (bpm)"]]